In [17]:
from scipy.optimize import nnls
from functools import partial

import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import ast
import torch
from sklearn.decomposition import NMF

from random import sample

from functions import cosine_perm,cosine_max
from AAUtoSig_init import AAUtoSig, train_AAUtoSig
from optuna_opt import optuna_tune

## Data management
Load the frequency matrix generated by the tool SomaticSiMu, and the set of signatures used for generating each instance. These are then matches by the patient ID, to determine what signatures are present in a given sub-sample

In [18]:
m = pd.read_csv(r'Q:\AUH-HAEM-FORSK-MutSigDLBCL222\article_1\generated_data\SomaticSiMu\Lung-AdenoCA_SBS96.csv')#.drop(['Unnamed: 0'], axis=1)

In [19]:
groundtruth = [line.strip("\n") for line in open(r'Q:\AUH-HAEM-FORSK-MutSigDLBCL222\article_1\generated_data\SomaticSiMu\Lung-AdenoCA_sbs_sigs.txt')]
groundtruth = pd.DataFrame([ast.literal_eval(x) for x in groundtruth])
groundtruth[0] = groundtruth[0].astype(int)


sig_names = np.unique(groundtruth.drop([0], axis = 1).fillna(":("))[1::]
COSMIC = pd.read_csv(r'COSMIC\COSMIC_v3.2_SBS_GRCh37.txt', sep = '\t', index_col=0)
sigs = COSMIC[sig_names]

In [20]:
xd = groundtruth[0].sort_values().tolist()
for i in range(len(xd)-1):
    if not (xd[i] + 1 == xd[i + 1]):
        print(xd[i])


25
33
161
361
533
565
597
629
661
692
713
721


## Split data
Firstly we write a method to split the mutational profile matrix into a training and validation set.

In [21]:
def split_data(data, frac_1):
  x_train = data.sample(frac = frac_1)
  x_validation = data.drop(x_train.index)
  return x_train, x_validation

# NMF performance

In [22]:
def out_errorNMF(train_df, validation_df, nsigs, true_sigs, max = False):
  model = NMF(n_components=nsigs, init='random', max_iter = 500)
  exposures = model.fit_transform(train_df)
  signatures = model.components_
  #cos_NMF = cosine_perm(true_sigs.to_numpy().T, signatures)[0]
  #cos_mean = np.mean(cos_NMF.diagonal())
  #cos_max = cosine_max(signatures, true_sigs.to_numpy().T)


  ref_exposures = model.transform(X = validation_df)
  rec = np.dot(ref_exposures, signatures)
  MSE = np.mean(((validation_df - rec)**2).to_numpy())
  
  if max == True:
    cos_max = cosine_max(signatures, true_sigs.to_numpy().T)
    return cos_max, MSE
  else:
    cos_NMF = cosine_perm(true_sigs.to_numpy().T, signatures)[0]
    cos_mean = np.mean(cos_NMF.diagonal())
    return cos_mean, MSE

# AAUtoSig performance

In [23]:
def out_error_AAUtoSig(train_df, validation_df, nsigs,true_sigs, max = False):
    model = AAUtoSig(nsigs)

    # Validation using MSE Loss function
    loss = torch.nn.MSELoss(reduction='mean')
    params = optuna_tune(train_df, nsigs, model_name = "AAUtoSig")

     # Using an Adam Optimizer with lr = 1e-3
    optimizer = torch.optim.Adam(model.parameters(),
                            lr = params['lr1'])
    train_AAUtoSig(500, model, train_df, loss_function= loss, optimizer=optimizer, batch_size = params['batch_size'])

    signatures = model.dec1.weight.data    
    signatures = pd.DataFrame(signatures.numpy())
    #asd = cosine_perm(true_sigs.to_numpy().T, signatures.T)
    #cos_Deep = asd[0]
    #cos_mean = np.mean(cos_Deep.diagonal())
    #cos_max = cosine_max(signatures.T, true_sigs.to_numpy().T)

    def calculate_exp(b):
       return(nnls(A = signatures, b = b)[0])
   
    ref_exposures = np.stack(validation_df.apply(calculate_exp, axis = 1))
    rec = np.dot(ref_exposures, signatures.T)

    MSE = np.mean(((validation_df - rec)**2).to_numpy())
    #return(MSE)
    if max == True:
       cos_max = cosine_max(signatures.T, true_sigs.to_numpy().T)
       return cos_max, MSE
    else:
       asd = cosine_perm(true_sigs.to_numpy().T, signatures.T)
       cos_Deep = asd[0]
       cos_mean = np.mean(cos_Deep.diagonal())
      
       return cos_mean, MSE

## Performance analysis

In [24]:
def performance_analysis(m, m_sigs):
  mut_matrix = (m.drop(['Unnamed: 0', '0'], axis = 1)).T.sample(400)
  train_data, test_data = split_data(mut_matrix, 0.75)
  
  idx_train = train_data.index
  idx_test = test_data.index
  y = [str(x) for x in m_sigs[0]]

  train_idx = list (set(idx_train) & set(y))
  test_idx = list(set(idx_test) & set(y))
  train_data = train_data[train_data.index.isin(train_idx)]
  test_data = test_data[test_data.index.isin(test_idx)]

  m_sigs_train = m_sigs[m_sigs[0].isin([int(x) for x in train_idx])]
  m_sigs_test = m_sigs[m_sigs[0].isin([int(x) for x in test_idx])]
  sig_names_train = np.unique(m_sigs_train.drop([0], axis = 1).fillna(":("))[1::]
  sig_names_test = np.unique(m_sigs_test.drop([0], axis = 1).fillna(":("))[1::]

  COSMIC = pd.read_csv(r'COSMIC\COSMIC_v3.1_SBS_GRCh38.txt', sep = '\t', index_col=0)
  sigs_train = COSMIC[sig_names_train]
  nsigs = len(sig_names_train)
  print("Hola!")
  return(out_errorNMF(train_data, test_data, nsigs, sigs_train, max = True) +
          out_errorNMF(train_data, test_data, nsigs, sigs_train) + 
          out_error_AAUtoSig(train_data, test_data, nsigs, sigs_train, max = True) +
           out_error_AAUtoSig(train_data, test_data, nsigs, sigs_train))

asd = [performance_analysis(m, groundtruth) for _ in range(8)] 
result = pd.DataFrame(asd)
result.columns = ["NMF_max", "outNMF1", "NMF_perm", "outNMF2",
                  "AE_max", "outAE1", "AE_perm", "outAE2" ]


Hola!


C:\Users\bjyw\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\sklearn\decomposition\_nmf.py:1637: ConvergenceWarning: Maximum number of iterations 500 reached. Increase it to improve convergence.
  warnings.warn(
C:\Users\bjyw\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\sklearn\decomposition\_nmf.py:1637: ConvergenceWarning: Maximum number of iterations 500 reached. Increase it to improve convergence.
  warnings.warn(


Hola!


C:\Users\bjyw\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\sklearn\decomposition\_nmf.py:1637: ConvergenceWarning: Maximum number of iterations 500 reached. Increase it to improve convergence.
  warnings.warn(
C:\Users\bjyw\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\sklearn\decomposition\_nmf.py:1637: ConvergenceWarning: Maximum number of iterations 500 reached. Increase it to improve convergence.
  warnings.warn(


Hola!


...

C:\Users\bjyw\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\sklearn\decomposition\_nmf.py:1637: ConvergenceWarning: Maximum number of iterations 500 reached. Increase it to improve convergence.
  warnings.warn(
C:\Users\bjyw\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\sklearn\decomposition\_nmf.py:1637: ConvergenceWarning: Maximum number of iterations 500 reached. Increase it to improve convergence.
  warnings.warn(


Hola!


C:\Users\bjyw\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\sklearn\decomposition\_nmf.py:1637: ConvergenceWarning: Maximum number of iterations 500 reached. Increase it to improve convergence.
  warnings.warn(
C:\Users\bjyw\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\sklearn\decomposition\_nmf.py:1637: ConvergenceWarning: Maximum number of iterations 500 reached. Increase it to improve convergence.
  warnings.warn(


Hola!


C:\Users\bjyw\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\sklearn\decomposition\_nmf.py:1637: ConvergenceWarning: Maximum number of iterations 500 reached. Increase it to improve convergence.
  warnings.warn(
C:\Users\bjyw\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\sklearn\decomposition\_nmf.py:1637: ConvergenceWarning: Maximum number of iterations 500 reached. Increase it to improve convergence.
  warnings.warn(


In [1]:
plt.scatter(y = result['NMF_perm'], x = result['NMF_max'], c = 'blue', label = 'NMF')
plt.scatter(y = result['AE_perm'], x = result['AE_max'], c = 'red', label = 'AE')
plt.xlabel('diagonal cosine - max value')
plt.ylabel('diagonal cosine - permutations')
plt.title('cosine values - max value')
plt.legend()
plt.show()

NameError: name 'plt' is not defined

In [26]:
plt.boxplot(result[['outNMF linear', 'outAE linear']], labels = ["NMF", "AE"])
plt.title('NMF vs AE out of sample error on linearly simulated data')
plt.show()

plt.boxplot(result[['cosineNMF linear', 'cosineAE linear']] ,labels = ["NMF", "AE"])
plt.title('NMF vs AE cosine values on linearly simulated data')
plt.show()

plt.scatter(y = result['cosineNMF linear'], x = result['outNMF linear'], c = 'blue', label = 'NMF')
plt.scatter(y = result['cosineAE linear'], x = result['outAE linear'], c = 'red', label = 'AE')
plt.xlabel('Out of sample error')
plt.ylabel('mean diagonal cosine')
plt.title('NMF vs AE performance on linearly simulated data')
plt.legend()
plt.show()
#plt.savefig('linear.png', transparent=True)

KeyError: "None of [Index(['outNMF linear', 'outAE linear'], dtype='object')] are in the [columns]"